In [1]:
import json
import numpy as np 
import pandas as pd
from loguru import logger
from modules import _get_driver_webscraping, _get_headers_webscraping, _number_of_pages, _get_html_with_many_cards, _read_results_parquet, _apply_functions_in_parallel_to_webscraping, _read_json_file, _extract_data_in_html_for_page_with_many_cards, _get_html_especifc_for_wich_property, _transform_data_to_dataframe

In [2]:
base_url = 'https://www.zapimoveis.com.br'
transaction = "aluguel"
type = "imoveis"
local = "rj+rio-de-janeiro"

In [3]:
# number_of_pages = _number_of_pages(base_url, transaction, type, local)
# print(number_of_pages)

number_of_pages = {'Parametros': {'Transacao': 'aluguel', 'Tipo': 'imoveis', 'Local': 'rj+rio-de-janeiro'}, 'Requisicao': {'Status': True}, 'Imoveis': 26981, 'Paginas': 269}
number_of_pages = json.dumps(number_of_pages, indent=4)
print(number_of_pages)


{
    "Parametros": {
        "Transacao": "aluguel",
        "Tipo": "imoveis",
        "Local": "rj+rio-de-janeiro"
    },
    "Requisicao": {
        "Status": true
    },
    "Imoveis": 26981,
    "Paginas": 269
}


In [1]:
# list_use_in_function = list(range(1, 269))
# function_to_apply = _get_html_with_many_cards
# name_file = 'urls_imoveis_aluguel'
# workers = 3
# args = [base_url, transaction, type, local]

# info_pages, erro_pages = _apply_functions_in_parallel_to_webscraping(list_use_in_function=list_use_in_function, function_to_apply=function_to_apply, name_file=name_file, key_name="url", workers=workers, args=args)
# info_pages, erro_pages

In [3]:
lista_informacoes_imoveis, erros_imoveis = _read_results_parquet(key_name="url", name_file="urls_imoveis_aluguel")
logger.success(f"Informações dos Imóveis Processadas: {len(lista_informacoes_imoveis)}")

2024-11-13 07:39:48.401 | SUCCESS  | __main__:<module>:2 - Informações dos Imóveis Processadas: 268


In [4]:
dataframe_urls = _extract_data_in_html_for_page_with_many_cards(lista_informacoes_imoveis)
dataframe_urls.head()
dataframe_urls = dataframe_urls.dropna(subset=['url_imovel'])
print(dataframe_urls.shape)
dataframe_urls.head()

(10064, 9)


,pagina,anuncio_pagina,id_imovel,url_imovel,destaque,bairro,endereco,descricao,data_completa
0,0,0,2741566013,https://www.zapimoveis.com.br/imovel/aluguel-c...,Super Destaque,"Laranjeiras, Rio de Janeiro",Rua Alice,"Aliança Viva aluga: Casa na Rua Alice, em Lara...",2024-11-13
1,0,1,2729441818,https://www.zapimoveis.com.br/imovel/aluguel-l...,Super Destaque,"Barra da Tijuca, Rio de Janeiro",Avenida das Américas,Ótima oportunidade para quem busca um espaço c...,2024-11-13
2,0,2,2750169530,https://www.zapimoveis.com.br/imovel/aluguel-a...,Super Destaque,"Lagoa, Rio de Janeiro",Avenida Epitácio Pessoa,"(Código: 36354, Avenida Epitácio Pessoa) - Loc...",2024-11-13
3,0,3,2755807416,https://www.zapimoveis.com.br/imovel/aluguel-c...,Destaque,"Copacabana, Rio de Janeiro",Rua Francisco Sá,Monte seu consultório/sala de pilates AQUI!\n\...,2024-11-13
5,0,5,2750819430,https://www.zapimoveis.com.br/imovel/venda-and...,NaN,"Centro, Rio de Janeiro",Avenida Marechal Floriano,"Centro, RJ. Conjunto comercial medindo aproxim...",2024-11-13


In [5]:
urls_de_imoveis = dataframe_urls['url_imovel'].to_list()
logger.success(f"URLs Processadas: {len(urls_de_imoveis)}")

2024-11-13 07:41:05.681 | SUCCESS  | __main__:<module>:2 - URLs Processadas: 10064


In [9]:
#list_use_in_function = urls_de_imoveis
#function_to_apply = _get_html_especifc_for_wich_property
#name_file = 'caracteristicas_imoveis_aluguel'
#workers = 3
#args = []

#info_pages, erro_pages = _apply_functions_in_parallel_to_webscraping(list_use_in_function=list_use_in_function, function_to_apply=function_to_apply, name_file=name_file, key_name="url", workers=workers, args=args)

In [11]:
lista_informacoes_imoveis_aluguel, erros_imoveis_aluguel = _read_results_parquet(key_name="url", name_file="caracteristicas_imoveis_aluguel")
logger.success(f"Informações dos Imóveis Processadas: {len(lista_informacoes_imoveis_aluguel)}")

2024-11-13 14:53:44.351 | SUCCESS  | __main__:<module>:2 - Informações dos Imóveis Processadas: 8322


In [17]:
def limpar_preco(valor):

    if isinstance(valor, str):
        valor = valor.replace("R$", "").replace(".", "").replace(",", ".").strip()
        try:
            valor = float(valor)
        except:
            valor = None
    
    return valor

def limpar_m2(valor):
    
    if isinstance(valor, str):
        valor = valor.replace("m²", "").replace(",", ".").strip()
        try:
            valor = float(valor)
        except:
            valor = None
    
    return valor

def limpar_numero(valor):
    if isinstance(valor, str):
        if "-" not in valor:
            valor = re.sub(r"[^0-9]", "", valor)  # Remove qualquer coisa que não seja número
            return int(valor) if valor else None
        else:
            valor = None
    return valor

dados_imoveis = pd.json_normalize(lista_informacoes_imoveis_aluguel)

# Identificar colunas com mais de X% de dados não nulos
dados_resumidos = dados_imoveis.loc[:, dados_imoveis.notnull().mean() > 0.4].copy()
dados_resumidos = dados_resumidos.dropna()
print(dados_resumidos.shape)
display(dados_resumidos.head())
#dados_resumidos = dados_resumidos.rename(columns={
#    "content.condominio": "condominio",
#    "content.endereco": "endereco",
#    "content.floorSize": "area",
#    "content.iptu": "iptu", 
#    "content.numberOfBathroomsTotal": "banheiros",
#    "content.numberOfParkingSpaces": "vagas_de_carro",
#    "content.numberOfRooms": "quartos",
#    "content.numberOfSuites": "suites",
#    "content.preco": "preco"
#})

#print(dados_resumidos.head())

# Aplica as funções de limpeza nas colunas
#dados_resumidos['preco'] = dados_resumidos['preco'].apply(limpar_preco)
#dados_resumidos['area'] = dados_resumidos['area'].apply(limpar_m2)
#dados_resumidos['condominio'] = dados_resumidos['condominio'].apply(limpar_preco)
#dados_resumidos['iptu'] = dados_resumidos['iptu'].apply(limpar_preco)
#dados_resumidos['banheiros'] = dados_resumidos['banheiros'].apply(limpar_numero)
#dados_resumidos['vagas_de_carro'] = dados_resumidos['vagas_de_carro'].apply(limpar_numero)
#dados_resumidos['quartos'] = dados_resumidos['quartos'].apply(limpar_numero)
#dados_resumidos['suites'] = dados_resumidos['suites'].apply(limpar_numero)

(2710, 9)


,url,content.condominio,content.endereco,content.floorSize,content.iptu,content.numberOfBathroomsTotal,content.numberOfParkingSpaces,content.numberOfRooms,content.preco
0,https://www.zapimoveis.com.br/imovel/aluguel-a...,R$ 1.982,"Avenida Epitácio Pessoa, 4344 - Lagoa, Rio de ...",137 m²,R$ 470,2 banheiros,2 vagas,3 quartos,R$ 12.500/mês
1,https://www.zapimoveis.com.br/imovel/aluguel-c...,não informado,"Rua Alice - Laranjeiras, Rio de Janeiro - RJ",300 m²,R$ 6.558,2 banheiros,4 vagas,8 quartos,R$ 20.000/mês
5,https://www.zapimoveis.com.br/imovel/aluguel-a...,R$ 700,"Avenida Salvador Allende, 931 - Barra da Tijuc...",50 m²,R$ 75,2 banheiros,1 vaga,2 quartos,R$ 2.300/mês
7,https://www.zapimoveis.com.br/imovel/aluguel-a...,R$ 409,"Rua Pinto Teles, 660 - Praça Seca, Rio de Jane...",60 m²,R$ 162,1 banheiro,1 vaga,2 quartos,R$ 1.100/mês
9,https://www.zapimoveis.com.br/imovel/aluguel-c...,R$ 100,"Rua Artur Rios, 541 - Senador Vasconcelos, Rio...",120 m²,R$ 42,4 banheiros,1 vaga,3 quartos,R$ 1.600/mês
